<a href="https://colab.research.google.com/github/MarcoBrac/Test/blob/master/test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
from google.colab import files

import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler


In [0]:
#drive.mount('/content/drive')

In [0]:
#!head TABLA_MODELO_HIT_3.csv

In [3]:
 uploaded = files.upload()

Saving TABLA_MODELO_HIT_3.csv to TABLA_MODELO_HIT_3.csv


In [0]:
df1 = pd.read_csv('TABLA_MODELO_HIT_3.csv' ) 

In [5]:
#df1.head()

,CONTRATO,NUM_CLIENTE,BC_SCORE,COSECHA,EDAD,OCPUACION_SEXO,CUENTA_MAS_ANTIGUA_BANCO,MAX_MOP_6M_AB_TDC,CUENTA_MAS_ANTIGUA_TDC,SALDO_VENCIDO_ABIERTAS,MAX_MOP_6M,PORC_UTILIZ_ENTRE_CUENT_REV_AB,PRODUCTO,SUB_PRODUCTO,VINTAGE,AMOUNT,FLAG_1PV_3,FLAG_2PV_6,FLAG_3PV_12,FLAG_3PV_18,FLAG_DESEMBOLSADO
0,400015808,1953462,660,201505,37.0,C_CON NEGOCIO/F,31.0,NaN,NaN,0.0,9.0,NaN,OMEGA,OMEGA_M,201505,11728.0,0,1,1,1,1
1,400015823,772865,663,201505,64.0,F_JUBILADO/F,94.0,2.0,187.0,511.0,2.0,0.219478,OMEGA,OMEGA_M,201505,17191.0,0,0,0,0,1
2,400015912,3873005,561,201505,41.0,A_CON NEGOCIO/M,28.0,NaN,NaN,4662.0,7.0,NaN,OMEGA,OMEGA_M,201505,5897.0,0,0,0,0,1
3,400015973,776366,724,201505,44.0,C_CON NEGOCIO/F,NaN,1.0,18.0,0.0,1.0,0.104229,OMEGA,OMEGA_M,201505,8000.0,0,0,0,0,1
4,400014820,347497,713,201505,40.0,C_CON NEGOCIO/F,49.0,1.0,42.0,0.0,1.0,0.034422,OMEGA,OMEGA_M,201505,29686.0,0,0,0,0,1


In [0]:
df2 = df1.copy()

In [0]:
df2 = df2[df2.COSECHA >= 201600]

In [0]:
df2 = df2[df2['EDAD'].isnull() == False]

In [0]:
df2['TARGET'] = df2.FLAG_3PV_18

In [0]:
#df2.head()

In [120]:
df2.columns

Index(['CONTRATO', 'NUM_CLIENTE', 'BC_SCORE', 'COSECHA', 'EDAD',
       'OCPUACION_SEXO', 'CUENTA_MAS_ANTIGUA_BANCO', 'MAX_MOP_6M_AB_TDC',
       'CUENTA_MAS_ANTIGUA_TDC', 'SALDO_VENCIDO_ABIERTAS', 'MAX_MOP_6M',
       'PORC_UTILIZ_ENTRE_CUENT_REV_AB', 'PRODUCTO', 'SUB_PRODUCTO', 'VINTAGE',
       'AMOUNT', 'FLAG_1PV_3', 'FLAG_2PV_6', 'FLAG_3PV_12', 'FLAG_3PV_18',
       'FLAG_DESEMBOLSADO', 'TARGET'],
      dtype='object')

In [0]:
vars_num = ['CUENTA_MAS_ANTIGUA_BANCO', 'MAX_MOP_6M_AB_TDC', 'CUENTA_MAS_ANTIGUA_TDC' , 'SALDO_VENCIDO_ABIERTAS', 'MAX_MOP_6M', 'PORC_UTILIZ_ENTRE_CUENT_REV_AB']
vars_fac = ['EDAD','OCPUACION_SEXO']

In [0]:
for v in vars_fac:
    aux = df2[[v,'TARGET']].copy()
    aux['n'] = 1
    aux = aux.pivot_table(index=v,columns='TARGET',aggfunc='count')
    aux.columns=aux.columns.droplevel()
    aux.reset_index(inplace=True)
    for i in range(2):
        aux[i]/=aux[i].sum()
    aux['woe'] =np.log(aux[0]/aux[1])
    df2 = df2.merge(aux[['woe',v]],how='inner',on=v)
    df2.rename(columns={'woe':'W_%s'%v},inplace=True)

In [123]:
df2.head(10)

,CONTRATO,NUM_CLIENTE,BC_SCORE,COSECHA,EDAD,OCPUACION_SEXO,CUENTA_MAS_ANTIGUA_BANCO,MAX_MOP_6M_AB_TDC,CUENTA_MAS_ANTIGUA_TDC,SALDO_VENCIDO_ABIERTAS,MAX_MOP_6M,PORC_UTILIZ_ENTRE_CUENT_REV_AB,PRODUCTO,SUB_PRODUCTO,VINTAGE,AMOUNT,FLAG_1PV_3,FLAG_2PV_6,FLAG_3PV_12,FLAG_3PV_18,FLAG_DESEMBOLSADO,TARGET,W_EDAD,W_OCPUACION_SEXO
0,400047237,2189555,662,201602,41.0,C_CON NEGOCIO/F,9.0,NaN,NaN,0.0,1.0,NaN,OMEGA,OMEGA_M,201602,13233.0,0,0,0,0,1,0,-0.316741,0.0199
1,400044595,1137075,700,201601,41.0,C_CON NEGOCIO/F,30.0,NaN,NaN,0.0,1.0,NaN,OMEGA,OMEGA_M,201601,18604.0,0,0,0,1,1,1,-0.316741,0.0199
2,400049282,1097485,674,201603,41.0,C_CON NEGOCIO/F,40.0,NaN,NaN,0.0,1.0,NaN,OMEGA,OMEGA_M,201603,17860.0,0,0,0,1,1,1,-0.316741,0.0199
3,400045142,1801043784,658,201601,41.0,C_CON NEGOCIO/F,NaN,NaN,NaN,0.0,2.0,NaN,OMEGA,OMEGA_M,201601,12924.0,0,0,0,0,1,0,-0.316741,0.0199
4,400048056,1800964301,606,201602,41.0,C_CON NEGOCIO/F,163.0,1.0,163.0,290.0,5.0,0.961808,OMEGA,OMEGA_M,201602,11290.0,0,0,0,1,1,1,-0.316741,0.0199
5,400044441,1022797,663,201601,41.0,C_CON NEGOCIO/F,73.0,NaN,NaN,0.0,1.0,NaN,OMEGA,OMEGA_M,201602,12323.0,0,0,0,0,1,0,-0.316741,0.0199
6,400043430,1077526,557,201601,41.0,C_CON NEGOCIO/F,93.0,5.0,38.0,2900.0,5.0,1.320200,OMEGA,OMEGA_M,201601,16022.0,0,0,0,0,1,0,-0.316741,0.0199
7,400044708,1800288502,657,201601,41.0,C_CON NEGOCIO/F,85.0,6.0,97.0,0.0,6.0,0.636667,OMEGA,OMEGA_M,201601,18502.0,0,0,0,0,1,0,-0.316741,0.0199
8,400049229,1801225207,646,201603,41.0,C_CON NEGOCIO/F,72.0,NaN,NaN,0.0,5.0,NaN,OMEGA,OMEGA_M,201603,10610.0,0,0,1,1,1,1,-0.316741,0.0199
9,400049985,1801750473,676,201603,41.0,C_CON NEGOCIO/F,36.0,NaN,NaN,0.0,1.0,NaN,OMEGA,OMEGA_M,201603,14249.0,0,0,0,0,1,0,-0.316741,0.0199


In [124]:
aux.head(10)

TARGET,OCPUACION_SEXO,0,1,woe
0,A_CON NEGOCIO/M,0.190003,0.290477,-0.424487
1,B_EMPLEADO O ASALARIADO/M,0.093966,0.100657,-0.068786
2,C_CON NEGOCIO/F,0.475007,0.465648,0.019900
3,D_EMPLEADO O ASALARIADO/F,0.128659,0.097626,0.276027
4,E_JUBILADO/M,0.039664,0.018565,0.759162
5,F_JUBILADO/F,0.072701,0.027027,0.989517


In [125]:
for i,v in enumerate(vars_num):
    bestk = 0
    best_iv = 0
    iv = 0
    for k in range(2,6):
  
        aux = df2[[v,'TARGET']].copy()
        aux[v] = pd.cut(aux[v],bins=k,include_lowest=True).astype(str)
        aux['n'] = 1
        aux = aux.pivot_table(index=v,columns='TARGET',aggfunc='count')
        aux.columns=aux.columns.droplevel()
        aux.reset_index(inplace=True)
        for i in range(2):
            aux[i]/=aux[i].sum()
        aux['woe'] =np.log(aux[0]/aux[1])
        aux['iv'] = aux['woe']*(aux[0]-aux[1])

        iv = aux['iv'].sum()

        print('%d %.6f'%(k,iv))

        if (aux['iv'].sum()> best_iv) & (~np.isinf(aux['iv'].sum())):
            best_iv = aux['iv'].sum()
            bestk = k


    aux = df2[[v,'TARGET']].copy()
    print("best %s %.4f %d"%(v,best_iv,bestk))
    aux[v] = pd.cut(aux[v],bins=bestk,include_lowest=True).astype(str)
    aux['n'] = 1
    aux = aux.pivot_table(index=v,columns='TARGET',aggfunc='count')
    aux.columns=aux.columns.droplevel()
    aux.reset_index(inplace=True)
    for i in range(2):
        aux[i]/=aux[i].sum()
    aux['woe'] =np.log(aux[0]/aux[1])
    df2[v] = pd.cut(df2[v],bins=bestk,include_lowest=True).astype(str)
    df2 = df2.merge(aux[['woe',v]],how='inner',on=v)
    df2.rename(columns={'woe':'W_%s'%v},inplace=True)


2 0.032622
3 0.055681
4 0.069109
5 0.102137
best CUENTA_MAS_ANTIGUA_BANCO 0.1021 5
2 0.100185
3 0.102539
4 0.112386
5 0.115887
best MAX_MOP_6M_AB_TDC 0.1159 5
2 0.098822
3 0.119889
4 0.122845
5 0.124583
best CUENTA_MAS_ANTIGUA_TDC 0.1246 5
2 0.000794
3 0.001076
4 0.002606
5 0.001848
best SALDO_VENCIDO_ABIERTAS 0.0026 4
2 0.041278
3 0.044383
4 0.054279
5 0.058689
best MAX_MOP_6M 0.0587 5
2 0.075072
3 0.075402
4 0.075150
5 0.075156
best PORC_UTILIZ_ENTRE_CUENT_REV_AB 0.0754 3


In [109]:
df2.CUENTA_MAS_ANTIGUA_BANCO.value_counts(normalize=True)

(-0.5, 99.8]      0.573226
(99.8, 199.6]     0.285068
nan               0.104186
(199.6, 299.4]    0.025972
(299.4, 399.2]    0.010329
(399.2, 499.0]    0.001220
Name: CUENTA_MAS_ANTIGUA_BANCO, dtype: float64

In [110]:
df2.MAX_MOP_6M_AB_TDC.value_counts(normalize=True)

(-0.009999999999999998, 1.8]    0.481538
nan                             0.369137
(1.8, 3.6]                      0.104240
(7.2, 9.0]                      0.031448
(3.6, 5.4]                      0.009434
(5.4, 7.2]                      0.004202
Name: MAX_MOP_6M_AB_TDC, dtype: float64

In [111]:
df2.CUENTA_MAS_ANTIGUA_TDC.value_counts(normalize=True)

(-0.613, 122.4]    0.516781
(122.4, 244.8]     0.221303
nan                0.209890
(244.8, 367.2]     0.042238
(367.2, 489.6]     0.009082
(489.6, 612.0]     0.000705
Name: CUENTA_MAS_ANTIGUA_TDC, dtype: float64

In [112]:
df2.SALDO_VENCIDO_ABIERTAS.value_counts(normalize=True)

(-463.25399999999996, 115813.25]    0.972618
nan                                 0.026379
(115813.25, 231626.5]               0.000813
(231626.5, 347439.75]               0.000108
(347439.75, 463253.0]               0.000081
Name: SALDO_VENCIDO_ABIERTAS, dtype: float64

In [113]:
df2.MAX_MOP_6M.value_counts(normalize=True)

(-0.009999999999999998, 1.8]    0.604457
(1.8, 3.6]                      0.222523
(7.2, 9.0]                      0.107087
(3.6, 5.4]                      0.029035
(5.4, 7.2]                      0.028819
nan                             0.008079
Name: MAX_MOP_6M, dtype: float64

In [114]:
df2.PORC_UTILIZ_ENTRE_CUENT_REV_AB.value_counts(normalize=True)

(-140.51500000000001, 46838.0]    0.633763
nan                               0.366101
(46838.0, 93676.0]                0.000108
(93676.0, 140514.0]               0.000027
Name: PORC_UTILIZ_ENTRE_CUENT_REV_AB, dtype: float64

In [0]:
############################### OCUPACIÓN SEXO
def bins_ocup_sex(row):
    if  row.OCPUACION_SEXO == 'A_CON NEGOCIO/M':
        return 1*0.438337084334697
    elif row.OCPUACION_SEXO == 'B_EMPLEADO O ASALARIADO/M':
        return 1*0.245752537229401
    elif  row.OCPUACION_SEXO == 'C_CON NEGOCIO/F':
        return 1*-0.0986946259109583
    elif  row.OCPUACION_SEXO == 'D_EMPLEADO O ASALARIADO/F':
        return 1*-0.289297648037313
    elif  row.OCPUACION_SEXO == 'E_JUBILADO/M':
        return 1*-0.859435727384404
    else:
        return 1*-1.06300066767039

In [0]:
###############################EDAD
def bins_edad(row):
    if  row.EDAD <= 39:
        return 1*0.652557304203474
    elif row.EDAD <= 46:
        return 1*0.32072210601685
    elif  row.EDAD <= 51:
        return 1*-0.0297882009277696
    elif  row.EDAD <= 56:
        return 1*-0.249830551211539
    elif  row.EDAD <= 61:
        return 1*-0.776458288161244
    else:
        return -0.862815250709056

In [0]:
############################### máxima antigÜedad banco 
def bins_max_mob_bank(row):
    if  row.CUENTA_MAS_ANTIGUA_BANCO <= 36:
        return 1*0.4177579842355
    elif row.CUENTA_MAS_ANTIGUA_BANCO <= 59:
        return 1*0.204288134959686
    elif  row.CUENTA_MAS_ANTIGUA_BANCO <= 76:
        return 1*0.079527302643976
    elif  row.CUENTA_MAS_ANTIGUA_BANCO <= 104:
        return 1*-0.221813867785179
    elif  row.CUENTA_MAS_ANTIGUA_BANCO <= 131:
        return 1*-0.474594512237555
    elif  row.CUENTA_MAS_ANTIGUA_BANCO <= 499:
        return 1*-0.730039690708382
    else:
        return 1*0.347094881317897

In [0]:
################################## pct utilización
def bins_pct_util(row):
    if  row.PORC_UTILIZ_ENTRE_CUENT_REV_AB <= 0.150290638:
        return 1*-0.510375236153387
    elif row.PORC_UTILIZ_ENTRE_CUENT_REV_AB <= 140514:
        return 1*-0.223185206563544
    else:
        return 1*0.434261584221614

In [0]:
######################################## MÁXIMO MOP TDC

def bins_max_mop_6m_ab_tdc(row):
    if  row.MAX_MOP_6M_AB_TDC <= 1:
        return 1*-0.530853505755427
    elif row.MAX_MOP_6M_AB_TDC <= 9:
        return 1*-0.0515609497107696
    else:
        return 1*0.462116439985472

In [0]:
############################### máxima antigÜedad TDC 
def bins_max_mob_tdc(row):
    if  row.CUENTA_MAS_ANTIGUA_TDC <= 53:
        return 1*0.0872105802631326
    elif row.CUENTA_MAS_ANTIGUA_TDC <= 98:
        return 1*-0.151627060166407
    elif  row.CUENTA_MAS_ANTIGUA_TDC <= 137:
        return 1*-0.352600764658328
    elif  row.CUENTA_MAS_ANTIGUA_TDC <= 612:
        return 1*-0.669246976557249
    else:
        return 1*0.606962693925261

In [0]:
############################## saldo vencido
def bins_sdo_vncido(row):
    if  row.SALDO_VENCIDO_ABIERTAS <= 1:
        return 1*-0.205208313977855
    elif row.SALDO_VENCIDO_ABIERTAS <= 463253:
        return 1*0.46669257203569
    else:
        return 1*0.470424547454944

In [0]:
############################## max mop 6m
def bins_max_mop_6m(row):
    if  row.MAX_MOP_6M <= 1:
        return 1*-0.236896747716894
    elif row.MAX_MOP_6M <= 2:
        return 1*-0.0605113857847795
    elif row.MAX_MOP_6M <= 9:
        return 1*0.523800086179919
    else:
        return 1*0.303489425193646

In [0]:
df2['edad_woe'] = df2.apply(bins_edad, axis=1)

In [0]:
df2['ocup_sex_woe'] = df2.apply(bins_ocup_sex ,axis=1)

In [0]:
df2['max_mob_bank_woe'] = df2.apply(bins_max_mob_bank, axis=1)

In [0]:
df2['pct_util_woe'] = df2.apply(bins_pct_util, axis=1)

In [0]:
df2['max_mop_6m_ab_tdc_woe'] = df2.apply(bins_max_mop_6m_ab_tdc, axis=1)

In [0]:
df2['max_mob_tdc_woe'] = df2.apply(bins_max_mob_tdc, axis=1)

In [0]:
df2['sdo_vncido_woe'] = df2.apply(bins_sdo_vncido, axis=1)

In [0]:
df2['max_mop_6m_woe'] = df2.apply(bins_max_mop_6m, axis=1)

In [0]:
df2.head(5)

,CONTRATO,NUM_CLIENTE,BC_SCORE,COSECHA,EDAD,OCPUACION_SEXO,CUENTA_MAS_ANTIGUA_BANCO,MAX_MOP_6M_AB_TDC,CUENTA_MAS_ANTIGUA_TDC,SALDO_VENCIDO_ABIERTAS,MAX_MOP_6M,PORC_UTILIZ_ENTRE_CUENT_REV_AB,PRODUCTO,SUB_PRODUCTO,VINTAGE,AMOUNT,FLAG_1PV_3,FLAG_2PV_6,FLAG_3PV_12,FLAG_3PV_18,FLAG_DESEMBOLSADO,edad_woe,ocup_sex_woe,max_mob_bank_woe,pct_util_woe,max_mop_6m_ab_tdc_woe,max_mob_tdc_woe,sdo_vncido_woe,max_mop_6m_woe,edad_bin,ocup_sex_bin,max_mob_bank_bin,pct_util_bin,max_mop_6m_ab_tdc_bin,max_mob_tdc_bin,sdo_vncido_bin,max_mop_6m_bin
2112,400047237,2189555,662,201602,41.0,C_CON NEGOCIO/F,9.0,NaN,NaN,0.0,1.0,NaN,OMEGA,OMEGA_M,201602,13233.0,0,0,0,0,1,0.320722,-0.098695,0.320722,0.320722,0.462116,0.606963,-0.205208,-0.236897,B_40_A_46,-0.098695,B_40_A_46,B_40_A_46,A_SIN_INFORMACION,A_SIN_INFORMACION,C_0_A_1,D_0_A_1
2113,400047294,1136954,708,201602,51.0,C_CON NEGOCIO/F,NaN,NaN,65.0,0.0,1.0,NaN,OMEGA,OMEGA_M,201602,20000.0,0,0,0,0,1,-0.029788,-0.098695,-0.029788,-0.029788,0.462116,-0.151627,-0.205208,-0.236897,C_47_A_51,-0.098695,C_47_A_51,C_47_A_51,A_SIN_INFORMACION,C_54_A_98,C_0_A_1,D_0_A_1
2121,400044473,2957012,683,201601,38.0,A_CON NEGOCIO/M,38.0,1.0,16.0,0.0,1.0,0.202786,OMEGA,OMEGA_M,201601,11892.0,0,0,0,0,1,0.652557,0.438337,0.652557,0.652557,-0.530854,0.087211,-0.205208,-0.236897,A_20_A_39,0.438337,A_20_A_39,A_20_A_39,C_0_A_1,B_0_A_53,C_0_A_1,D_0_A_1
2122,400044567,1800814618,691,201601,31.0,C_CON NEGOCIO/F,84.0,NaN,84.0,0.0,1.0,NaN,OMEGA,OMEGA_M,201601,10968.0,0,1,1,1,1,0.652557,-0.098695,0.652557,0.652557,0.462116,-0.151627,-0.205208,-0.236897,A_20_A_39,-0.098695,A_20_A_39,A_20_A_39,A_SIN_INFORMACION,C_54_A_98,C_0_A_1,D_0_A_1
2123,400044630,1093288,593,201601,54.0,C_CON NEGOCIO/F,150.0,1.0,150.0,0.0,4.0,0.794667,OMEGA,OMEGA_M,201602,13615.0,0,0,0,0,1,-0.249831,-0.098695,-0.249831,-0.249831,-0.530854,-0.669247,-0.205208,0.523800,D_52_A_56,-0.098695,D_52_A_56,D_52_A_56,C_0_A_1,E_138_A_612,C_0_A_1,A_3_9


##**RANGOS**

In [0]:
############################### máxima antigÜedad banco 
def rango_max_mob_bank(row):
    if  row.CUENTA_MAS_ANTIGUA_BANCO <= 36:
        return 'A_0_A_36'
    elif row.CUENTA_MAS_ANTIGUA_BANCO <= 59:
        return 'C_37_A_59'
    elif  row.CUENTA_MAS_ANTIGUA_BANCO <= 76:
        return 'D_60_A_76'
    elif  row.CUENTA_MAS_ANTIGUA_BANCO <= 104:
        return 'E_77_A_104'
    elif  row.CUENTA_MAS_ANTIGUA_BANCO <= 131:
        return 'F_105_A_131'
    elif  row.CUENTA_MAS_ANTIGUA_BANCO <= 499:
        return 'G_132_A_499'
    else:
        return 'B_SIN_INFORMACION'

In [0]:
############################### OCUPACIÓN SEXO
def rango_ocup_sex(row):
    if  row.OCPUACION_SEXO == 'A_CON NEGOCIO/M':
        return 'A_CON NEGOCIO/M'
    elif row.OCPUACION_SEXO == 'B_EMPLEADO O ASALARIADO/M':
        return 'B_EMPLEADO O ASALARIADO/M'
    elif  row.OCPUACION_SEXO == 'C_CON NEGOCIO/F':
        return 'C_CON NEGOCIO/F'
    elif  row.OCPUACION_SEXO == 'D_EMPLEADO O ASALARIADO/F':
        return 'D_EMPLEADO O ASALARIADO/F'
    elif  row.OCPUACION_SEXO == 'E_JUBILADO/M':
        return 'E_JUBILADO/M'
    else:
        return 'F_JUBILADO/F'

In [0]:
###############################EDAD
def rango_edad(row):
    if  row.EDAD <= 39:
        return 'A_20_A_39'
    elif row.EDAD <= 46:
        return 'B_40_A_46'
    elif  row.EDAD <= 51:
        return 'C_47_A_51'
    elif  row.EDAD <= 56:
        return 'D_52_A_56'
    elif  row.EDAD <= 61:
        return 'E_57_A_61'
    else:
        return 'F_62_A_86'

In [0]:
################################## pct utilización
def rango_pct_util(row):
    if  row.PORC_UTILIZ_ENTRE_CUENT_REV_AB <= 0.150290638:
        return 'C_0_A_15'
    elif row.PORC_UTILIZ_ENTRE_CUENT_REV_AB <= 140514:
        return 'B_MAYOR_A_15'
    else:
        return 'A_SIN_INFORMACION'

In [0]:
######################################## MÁXIMO MOP TDC

def rango_max_mop_6m_ab_tdc(row):
    if  row.MAX_MOP_6M_AB_TDC <= 1:
        return 'C_0_A_1'
    elif row.MAX_MOP_6M_AB_TDC <= 9:
        return 'B_2_A_9'
    else:
        return 'A_SIN_INFORMACION'

In [0]:
############################### máxima antigÜedad TDC 
def rango_max_mob_tdc(row):
    if  row.CUENTA_MAS_ANTIGUA_TDC <= 53:
        return 'B_0_A_53'
    elif row.CUENTA_MAS_ANTIGUA_TDC <= 98:
        return 'C_54_A_98'
    elif  row.CUENTA_MAS_ANTIGUA_TDC <= 137:
        return 'D_99_A_137'
    elif  row.CUENTA_MAS_ANTIGUA_TDC <= 612:
        return 'E_138_A_612'
    else:
        return 'A_SIN_INFORMACION'

In [0]:
############################## saldo vencido
def rango_sdo_vncido(row):
    if  row.SALDO_VENCIDO_ABIERTAS <= 1:
        return 'C_0_A_1'
    elif row.SALDO_VENCIDO_ABIERTAS <= 463253:
        return 'B_2+'
    else:
        return 'A_SIN_INFORMACION'

In [0]:
############################## max mop 6m
def rango_max_mop_6m(row):
    if  row.MAX_MOP_6M <= 1:
        return 'D_0_A_1'
    elif row.MAX_MOP_6M <= 2:
        return 'C_2'
    elif row.MAX_MOP_6M <= 9:
        return 'A_3_9'
    else:
        return 'B_SIN_INFORMACION'

In [0]:
df2['edad_bin'] = df2.apply(rango_edad, axis=1)

In [0]:
df2['ocup_sex_bin'] = df2.apply(rango_ocup_sex ,axis=1)

In [0]:
df2['max_mob_bank_bin'] = df2.apply(rango_max_mob_bank, axis=1)

In [0]:
df2['pct_util_bin'] = df2.apply(rango_pct_util, axis=1)

In [0]:
df2['max_mop_6m_ab_tdc_bin'] = df2.apply(rango_max_mop_6m_ab_tdc, axis=1)

In [0]:
df2['max_mob_tdc_bin'] = df2.apply(rango_max_mob_tdc, axis=1)

In [0]:
df2['sdo_vncido_bin'] = df2.apply(rango_sdo_vncido, axis=1)

In [0]:
df2['max_mop_6m_bin'] = df2.apply(rango_max_mop_6m, axis=1)

In [0]:
df2.head()

,CONTRATO,NUM_CLIENTE,BC_SCORE,COSECHA,EDAD,OCPUACION_SEXO,CUENTA_MAS_ANTIGUA_BANCO,MAX_MOP_6M_AB_TDC,CUENTA_MAS_ANTIGUA_TDC,SALDO_VENCIDO_ABIERTAS,MAX_MOP_6M,PORC_UTILIZ_ENTRE_CUENT_REV_AB,PRODUCTO,SUB_PRODUCTO,VINTAGE,AMOUNT,FLAG_1PV_3,FLAG_2PV_6,FLAG_3PV_12,FLAG_3PV_18,FLAG_DESEMBOLSADO,edad_woe,ocup_sex_woe,max_mob_bank_woe,pct_util_woe,max_mop_6m_ab_tdc_woe,max_mob_tdc_woe,sdo_vncido_woe,max_mop_6m_woe,edad_bin,ocup_sex_bin,max_mob_bank_bin,pct_util_bin,max_mop_6m_ab_tdc_bin,max_mob_tdc_bin,sdo_vncido_bin,max_mop_6m_bin
2112,400047237,2189555,662,201602,41.0,C_CON NEGOCIO/F,9.0,NaN,NaN,0.0,1.0,NaN,OMEGA,OMEGA_M,201602,13233.0,0,0,0,0,1,0.320722,-0.098695,0.417758,0.434262,0.462116,0.606963,-0.205208,-0.236897,B_40_A_46,C_CON NEGOCIO/F,A_0_A_36,A_SIN_INFORMACION,A_SIN_INFORMACION,A_SIN_INFORMACION,C_0_A_1,D_0_A_1
2113,400047294,1136954,708,201602,51.0,C_CON NEGOCIO/F,NaN,NaN,65.0,0.0,1.0,NaN,OMEGA,OMEGA_M,201602,20000.0,0,0,0,0,1,-0.029788,-0.098695,0.347095,0.434262,0.462116,-0.151627,-0.205208,-0.236897,C_47_A_51,C_CON NEGOCIO/F,B_SIN_INFORMACION,A_SIN_INFORMACION,A_SIN_INFORMACION,C_54_A_98,C_0_A_1,D_0_A_1
2121,400044473,2957012,683,201601,38.0,A_CON NEGOCIO/M,38.0,1.0,16.0,0.0,1.0,0.202786,OMEGA,OMEGA_M,201601,11892.0,0,0,0,0,1,0.652557,0.438337,0.204288,-0.223185,-0.530854,0.087211,-0.205208,-0.236897,A_20_A_39,A_CON NEGOCIO/M,C_37_A_59,B_MAYOR_A_15,C_0_A_1,B_0_A_53,C_0_A_1,D_0_A_1
2122,400044567,1800814618,691,201601,31.0,C_CON NEGOCIO/F,84.0,NaN,84.0,0.0,1.0,NaN,OMEGA,OMEGA_M,201601,10968.0,0,1,1,1,1,0.652557,-0.098695,-0.221814,0.434262,0.462116,-0.151627,-0.205208,-0.236897,A_20_A_39,C_CON NEGOCIO/F,E_77_A_104,A_SIN_INFORMACION,A_SIN_INFORMACION,C_54_A_98,C_0_A_1,D_0_A_1
2123,400044630,1093288,593,201601,54.0,C_CON NEGOCIO/F,150.0,1.0,150.0,0.0,4.0,0.794667,OMEGA,OMEGA_M,201602,13615.0,0,0,0,0,1,-0.249831,-0.098695,-0.730040,-0.223185,-0.530854,-0.669247,-0.205208,0.523800,D_52_A_56,C_CON NEGOCIO/F,G_132_A_499,B_MAYOR_A_15,C_0_A_1,E_138_A_612,C_0_A_1,A_3_9


In [0]:
df2.to_csv('modelo_hit_4_woe')

In [0]:
df2.groupby('edad_bin').aggregate({'edad_woe' : {'_n_' : 'count' 
                                    ,'min_woe' : np.min
                                    ,'max_woe' : np.max
                                    ,'mean'    : np.mean}})

/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


edad_woe                              
               _n_   min_woe   max_woe      mean
edad_bin                                        
A_20_A_39     6172  0.652557  0.652557  0.652557
B_40_A_46     7206  0.320722  0.320722  0.320722
C_47_A_51     5997 -0.029788 -0.029788 -0.029788
D_52_A_56     6224 -0.249831 -0.249831 -0.249831
E_57_A_61     5219 -0.776458 -0.776458 -0.776458
F_62_A_86     6068 -0.862815 -0.862815 -0.862815

In [0]:
df2.groupby('ocup_sex_bin').aggregate({'ocup_sex_woe' : {'_n_' : 'count' 
                                    ,'min_woe' : np.min
                                    ,'max_woe' : np.max
                                    ,'mean'    : np.mean}})

/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


ocup_sex_woe                              
                                   _n_   min_woe   max_woe      mean
ocup_sex_bin                                                        
A_CON NEGOCIO/M                   7804  0.438337  0.438337  0.438337
B_EMPLEADO O ASALARIADO/M         3519  0.245753  0.245753  0.245753
C_CON NEGOCIO/F                  17447 -0.098695 -0.098695 -0.098695
D_EMPLEADO O ASALARIADO/F         4500 -0.289298 -0.289298 -0.289298
E_JUBILADO/M                      1296 -0.859436 -0.859436 -0.859436
F_JUBILADO/F                      2320 -1.063001 -1.063001 -1.063001

In [0]:
df2.groupby('max_mob_bank_bin').aggregate({'max_mob_bank_woe' : {'_n_' : 'count' 
                                    ,'min_woe' : np.min
                                    ,'max_woe' : np.max
                                    ,'mean'    : np.mean}})

/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


max_mob_bank_woe                              
                               _n_   min_woe   max_woe      mean
max_mob_bank_bin                                                
A_0_A_36                      5732  0.417758  0.417758  0.417758
B_SIN_INFORMACION             3843  0.347095  0.347095  0.347095
C_37_A_59                     5417  0.204288  0.204288  0.204288
D_60_A_76                     5478  0.079527  0.079527  0.079527
E_77_A_104                    5430 -0.221814 -0.221814 -0.221814
F_105_A_131                   5617 -0.474595 -0.474595 -0.474595
G_132_A_499                   5369 -0.730040 -0.730040 -0.730040

In [0]:
df2.groupby('pct_util_bin').aggregate({'pct_util_woe' : {'_n_' : 'count' 
                                    ,'min_woe' : np.min
                                    ,'max_woe' : np.max
                                    ,'mean'    : np.mean}})

/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


pct_util_woe                              
                           _n_   min_woe   max_woe      mean
pct_util_bin                                                
A_SIN_INFORMACION        13504  0.434262  0.434262  0.434262
B_MAYOR_A_15             11691 -0.223185 -0.223185 -0.223185
C_0_A_15                 11691 -0.510375 -0.510375 -0.510375

In [0]:
df2.groupby('max_mop_6m_ab_tdc_bin').aggregate({'max_mop_6m_ab_tdc_woe' : {'_n_' : 'count' 
                                    ,'min_woe' : np.min
                                    ,'max_woe' : np.max
                                    ,'mean'    : np.mean}})

/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


max_mop_6m_ab_tdc_woe                              
                                        _n_   min_woe   max_woe      mean
max_mop_6m_ab_tdc_bin                                                    
A_SIN_INFORMACION                     13616  0.462116  0.462116  0.462116
B_2_A_9                                5508 -0.051561 -0.051561 -0.051561
C_0_A_1                               17762 -0.530854 -0.530854 -0.530854

In [0]:
df2.groupby('max_mob_tdc_bin').aggregate({'max_mob_tdc_woe' : {'_n_' : 'count' 
                                    ,'min_woe' : np.min
                                    ,'max_woe' : np.max
                                    ,'mean'    : np.mean}})

/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


max_mob_tdc_woe                              
                              _n_   min_woe   max_woe      mean
max_mob_tdc_bin                                                
A_SIN_INFORMACION            7742  0.606963  0.606963  0.606963
B_0_A_53                     7315  0.087211  0.087211  0.087211
C_54_A_98                    7366 -0.151627 -0.151627 -0.151627
D_99_A_137                   7213 -0.352601 -0.352601 -0.352601
E_138_A_612                  7250 -0.669247 -0.669247 -0.669247

In [0]:
df2.groupby('sdo_vncido_bin').aggregate({'sdo_vncido_woe' : {'_n_' : 'count' 
                                    ,'min_woe' : np.min
                                    ,'max_woe' : np.max
                                    ,'mean'    : np.mean}})

/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


sdo_vncido_woe                              
                             _n_   min_woe   max_woe      mean
sdo_vncido_bin                                                
A_SIN_INFORMACION            973  0.470425  0.470425  0.470425
B_2+                        8067  0.466693  0.466693  0.466693
C_0_A_1                    27846 -0.205208 -0.205208 -0.205208

In [0]:
df2.groupby('max_mop_6m_bin').aggregate({'max_mop_6m_woe' : {'_n_' : 'count' 
                                    ,'min_woe' : np.min
                                    ,'max_woe' : np.max
                                    ,'mean'    : np.mean}})

/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


max_mop_6m_woe                              
                             _n_   min_woe   max_woe      mean
max_mop_6m_bin                                                
A_3_9                       7598  0.523800  0.523800  0.523800
B_SIN_INFORMACION            298  0.303489  0.303489  0.303489
C_2                         6694 -0.060511 -0.060511 -0.060511
D_0_A_1                    22296 -0.236897 -0.236897 -0.236897

In [0]:
#####REGRESIÓN 

In [0]:
df2.columns

Index(['CONTRATO', 'NUM_CLIENTE', 'BC_SCORE', 'COSECHA', 'EDAD',
       'OCPUACION_SEXO', 'CUENTA_MAS_ANTIGUA_BANCO', 'MAX_MOP_6M_AB_TDC',
       'CUENTA_MAS_ANTIGUA_TDC', 'SALDO_VENCIDO_ABIERTAS', 'MAX_MOP_6M',
       'PORC_UTILIZ_ENTRE_CUENT_REV_AB', 'PRODUCTO', 'SUB_PRODUCTO', 'VINTAGE',
       'AMOUNT', 'FLAG_1PV_3', 'FLAG_2PV_6', 'FLAG_3PV_12', 'FLAG_3PV_18',
       'FLAG_DESEMBOLSADO', 'edad_woe', 'ocup_sex_woe', 'max_mob_bank_woe',
       'pct_util_woe', 'max_mop_6m_ab_tdc_woe', 'max_mob_tdc_woe',
       'sdo_vncido_woe', 'max_mop_6m_woe'],
      dtype='object')

In [0]:
##### x
x_n = df2[['edad_woe', 'ocup_sex_woe', 'max_mob_bank_woe','pct_util_woe', 'max_mop_6m_ab_tdc_woe', 'max_mob_tdc_woe','sdo_vncido_woe', 'max_mop_6m_woe']]

In [0]:
##### y 
y_ = df2[['FLAG_3PV_18']]

In [0]:
test1 = LogisticRegression(fit_intercept = True)

In [0]:
test1.fit(x_n , y_)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
test1.coef_.tolist()

[[0.20263429514433537,
  0.6935454941373286,
  0.20263429514433537,
  0.20263429514433537,
  0.3185156960963072,
  0.5421581641972483,
  0.36182021869310543,
  0.638545703983806]]

In [0]:
x_n.columns.tolist()


['edad_woe',
 'ocup_sex_woe',
 'max_mob_bank_woe',
 'pct_util_woe',
 'max_mop_6m_ab_tdc_woe',
 'max_mob_tdc_woe',
 'sdo_vncido_woe',
 'max_mop_6m_woe']

In [0]:
coefs_ = pd.DataFrame(test1.coef_.tolist(), columns = x_n.columns.tolist() )

In [0]:
coefs_

,edad_woe,ocup_sex_woe,max_mob_bank_woe,pct_util_woe,max_mop_6m_ab_tdc_woe,max_mob_tdc_woe,sdo_vncido_woe,max_mop_6m_woe
0,0.202634,0.693545,0.202634,0.202634,0.318516,0.542158,0.36182,0.638546


#**RANGOS**

In [0]:
############################### OCUPACIÓN SEXO
def bins_ocup_sex(row):
    if  row.OCPUACION_SEXO == 'A_CON NEGOCIO/M':
        return 'A_CON NEGOCIO/M'
    elif row.OCPUACION_SEXO == 'B_EMPLEADO O ASALARIADO/M':
        return 'B_EMPLEADO O ASALARIADO/M'
    elif  row.OCPUACION_SEXO == 'C_CON NEGOCIO/F':
        return 'C_CON NEGOCIO/F'
    elif  row.OCPUACION_SEXO == 'D_EMPLEADO O ASALARIADO/F':
        return 'D_EMPLEADO O ASALARIADO/F'
    elif  row.OCPUACION_SEXO == 'E_JUBILADO/M':
        return 'E_JUBILADO/M'
    else:
        return 'F_JUBILADO/F'

In [0]:
###############################EDAD
def bins_edad(row):
    if  row.EDAD <= 39:
        return 'A_20_A_39'
    elif row.EDAD <= 46:
        return 'B_40_A_46'
    elif  row.EDAD <= 51:
        return 'C_47_A_51'
    elif  row.EDAD <= 56:
        return 'D_52_A_56'
    elif  row.EDAD <= 61:
        return 'E_57_A_61'
    else:
        return 'F_62_A_86'

In [0]:
############################### máxima antigÜedad banco 
def bins_max_mob_bank(row):
    if  row.CUENTA_MAS_ANTIGUA_BANCO <= 36:
        return 'A_0_A_36'
    elif row.CUENTA_MAS_ANTIGUA_BANCO <= 59:
        return 'C_37_A_59'
    elif  row.CUENTA_MAS_ANTIGUA_BANCO <= 76:
        return 'D_60_A_76'
    elif  row.CUENTA_MAS_ANTIGUA_BANCO <= 104:
        return 'E_77_A_104'
    elif  row.CUENTA_MAS_ANTIGUA_BANCO <= 131:
        return 'F_105_A_131'
    elif  row.CUENTA_MAS_ANTIGUA_BANCO <= 499:
        return 'G_132_A_499'
    else:
        return 'B_SIN_INFORMACION'

In [0]:
################################## pct utilización
def bins_pct_util(row):
    if  row.PORC_UTILIZ_ENTRE_CUENT_REV_AB <= 0.150290638:
        return 'C_0_A_15'
    elif row.PORC_UTILIZ_ENTRE_CUENT_REV_AB <= 140514:
        return 'B_MAYOR_A_15'
    else:
        return 'A_SIN_INFORMACION'

In [0]:
######################################## MÁXIMO MOP TDC

def bins_max_mop_6m_ab_tdc(row):
    if  row.MAX_MOP_6M_AB_TDC <= 1:
        return 'C_0_A_1'
    elif row.MAX_MOP_6M_AB_TDC <= 9:
        return 'B_2_A_9'
    else:
        return 'A_SIN_INFORMACION'

In [0]:
############################### máxima antigÜedad TDC 
def bins_max_mob_tdc(row):
    if  row.CUENTA_MAS_ANTIGUA_TDC <= 53:
        return 'B_0_A_53'
    elif row.CUENTA_MAS_ANTIGUA_TDC <= 98:
        return 'C_54_A_98'
    elif  row.CUENTA_MAS_ANTIGUA_TDC <= 137:
        return 'D_99_A_137'
    elif  row.CUENTA_MAS_ANTIGUA_TDC <= 612:
        return 'E_138_A_612'
    else:
        return 'A_SIN_INFORMACION'

In [0]:
############################## saldo vencido
def bins_sdo_vncido(row):
    if  row.SALDO_VENCIDO_ABIERTAS <= 1:
        return 'C_0_A_1'
    elif row.SALDO_VENCIDO_ABIERTAS <= 463253:
        return 'B_2+'
    else:
        return 'A_SIN_INFORMACION'

In [0]:
############################## max mop 6m
def bins_max_mop_6m(row):
    if  row.MAX_MOP_6M <= 1:
        return 'D_0_A_1'
    elif row.MAX_MOP_6M <= 2:
        return 'C_2'
    elif row.MAX_MOP_6M <= 9:
        return 'A_3_9'
    else:
        return 'B_SIN_INFORMACION'